In [ ]:
import json
import os
import pprint
import shutil
from collections import defaultdict, Counter
from library_dicom.dicom_processor.tools.folders import write_json_file
from library_dicom.dicom_processor.tools.cleaning_series import *

In [ ]:
liste_json = os.listdir('/media/deeplearning/VERBATIM HD/LNH073B_JSON/JSON_2')
merged_content = generate_merged_file('/media/deeplearning/VERBATIM HD/LNH073B_JSON/JSON_2')
write_json_file('/media/deeplearning/VERBATIM HD/LNH073B_JSON/JSON_2', 'merged_files', merged_content)

In [ ]:
non_interesting_series  = find_non_intersting_series('/media/deeplearning/VERBATIM HD/LNH073B_JSON/JSON_2/merged_files.json')
print(len(non_interesting_series))

In [ ]:
for non_interisting in non_interesting_series:
    try : 
        shutil.rmtree(non_interisting)
    except Exception as err : 
        print(err)

In [ ]:
series_ready, paths_series_ready = find_studies_with_two_series('/media/deeplearning/VERBATIM HD/LNH073B_JSON/JSON_2/merged_files.json')
print(len(series_ready))
print(len(paths_series_ready))

In [ ]:
paths_series_ready

In [ ]:
studies_over_two_series, paths = find_studies_over_two_series('/media/deeplearning/VERBATIM HD/LNH073B_JSON/JSON_2/merged_files.json')
print(len(studies_over_two_series))
print(len(paths))
#print(Counter(studies_over_two_series))

In [ ]:
import csv
csv_path= '/media/deeplearning/VERBATIM HD/LNH073B.csv'
with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    dataset = []
    for row in reader :
        dataset.append(row)
        
del dataset[0] #enlever première ligne

In [ ]:
#copy studies with more than 2 series to check 
from library_dicom.dicom_processor.model.Series import Series
export_folder = '/media/deeplearning/VERBATIM HD/LNH073B_check_serie'
for path in paths : 
    try : 
        print(path)
        details = Series(path).get_series_details()
        study_uid = details['study']['StudyInstanceUID']
        serie_uid = details['series']['SeriesInstanceUID']
        print(study_uid)
        print(serie_uid)
        for row in dataset : 
            if row[2] == study_uid and row[3]==serie_uid : 
                patient_id = row[1]
        print(patient_id)
        #dest = '/'.join(path.split('/')[-3:])
        #print(dest)
        destination = export_folder+'/'+patient_id+'/'+study_uid+'/'+serie_uid
        print(destination)
        os.makedirs(destination, exist_ok=True)
        
        shutil.move(path, destination)
    except Exception as err : 
        print(err)
        

In [ ]:
export_folder = '/media/deeplearning/VERBATIM HD/LNH073B_Validated_DICOM'
for path in paths_series_ready : 
    try : 
        print(path)
        details = Series(path).get_series_details()
        study_uid = details['study']['StudyInstanceUID']
        serie_uid = details['series']['SeriesInstanceUID']
        #print(study_uid)
        #print(serie_uid)
        for row in dataset : 
            if row[2] == study_uid and row[3]==serie_uid : 
                patient_id = row[1].split(' ')[0]
        #print(patient_id)
        #dest = '/'.join(path.split('/')[-3:])
        #print(dest)
        destination = export_folder+'/'+patient_id+'/'+study_uid+'/'+serie_uid
        print(destination)
        os.makedirs(destination, exist_ok=True)
        
        shutil.move(path, destination)
    except Exception as err : 
        print(err)

In [ ]:
import shutil 
root_destination = '/media/oncopole/DD 2To/FLIP/FLIP_Validated_DICOM'
for seriesID in series_ready : 
    try : 
        source_path = series_ready[seriesID]['path']
        #print(source_path)
        parentPatientID = series_ready[seriesID]['parentPatientID']
        parentStudyUID = series_ready[seriesID]['parentStudyUID']
        
        #print(type_)
        destination = root_destination+"/"+parentPatientID+"/"+"pet0"+"/"+seriesID
        print(destination)
        os.makedirs(destination, exist_ok=True)
        #copy and not move 
        #shutil.copytree(source_path, destination)
        shutil.move(source_path, destination)
    except Exception as err : 
        print(err)